In [5]:
import pandas as pd
import numpy as np
import openpyxl
import os
import sys
import msoffcrypto


def make_dataset(hyperparam):
    """ 各部のデータを結合する関数
    
    """

    # データ前処理に関する情報読み込み
    data_process_dict = hyperparam["data_processer"]

    # 各キー (外債部, 債券部) に対してループ
    for category, settings in data_process_dict.items():
        print(f"{category} データの処理開始")

        # すべてのフォルダのデータを格納する辞書
        all_folders_data = {}
        # フォルダパス
        folder_path = settings["folder_path"]

        # 各部署の最上位フォルダ内のサブフォルダに対して処理を実行
        for folder in os.listdir(folder_path):
            # サブフォルダのパスを取得
            sub_folder_path = os.path.join(folder_path, folder)
            if os.path.isdir(sub_folder_path):
                # processer_folder 関数を用いて各シートのデータを結合
                folder_data = processer_folder(settings, sub_folder_path)
            # 作成した辞書 (folder_data) より，データ結合
            for sheet_name, df_list in folder_data:
                if sheet_name not in all_folder_data:
                    all_folder_data[sheet_name] = []
                # 追加
                all_folder_data[sheet_name].extend(df_list)

        # 各シートのデータフレームを結合
        combined_dfs = {sheet: pd.concat(all_folders_data[sheet]) for sheet in all_folders_data}

        

def process_folder(settings, folder_path):
    # シート名ごとにデータフレームを格納する辞書
    all_sheets_data = {}
    # 指定したフォルダのファイルをリストで取得
    files = [f for f in os.listdir(folder_path) if f.endswith(settings["extension"])]

    # ファイルリストにあるファイルを開く
    for file in files:
        # ファイルパスを取得
        file_path = os.path.join(folder_path, file)
        with open(file_path, "rb") as f:
            office_file = msoffcrypto.OfficeFile(f)
            office_file.load_key(password = settings["password"])
            # ファイル一時的に保存
            with open("temp.xlsx", "wb") as temp_f:
                office_file.decrypt(temp_f)

        # すべてのシート名を取得
        xls = pd.ExcelFile("temp.xlsx")
        for sheet_name in xls.sheet_names:
            df = pd.read_excel("temp.xlsx", sheet_name = sheet_name)

            # 各シートのデータを追加
            if sheet_name not in all_sheets_data:
                all_sheets_data[sheet_name] = []
            all_sheets_data[sheet_name].append(df)
    
    os.remove("temp.xlsx")

    # 各シートのデータフレームを結合
    combined_dfs = {}
    # 辞書 (all_sheets_data) のアイテムを結合
    for sheet, dfs in all_sheets_data.items():
        combined_df = pd.concat(dfs)
        if sheet not in combined_dfs:
            combined_dfs[sheet] = combined_df
        else:
            # 主キーを用いてデータフレームを結合
            combined_dfs[sheet] = pd.merge(combined_dfs[sheet], combined_df, on = settings["primary_keys"], how = 'outer')

    return combined_dfs



if __name__ == "__main__":
    # データ前処理に関する設定取得
    from config.config import hyperparam
    # フォルダ作成
    make_dataset(hyperparam)

In [1]:
from config.config import hyperparam
# データ前処理に関する情報読み込み
data_process_dict = hyperparam["data_processer"]

# 各キー (外債部, 債券部) に対してループ
for category, settings in data_process_dict.items():
    print(f"処理カテゴリ: {category}")
    for key, value in settings.items():
        print(f"{key} : {value}")

処理カテゴリ: foreign_bond
folder_path : 
password : daiichilife
principle_key : ['Date, ISIN']
extension : .xlsx
処理カテゴリ: domestic_bond
folder_path : 
password : daiichilife
principle_key : ['Date, ISIN']
extension : .xlsx


In [ ]:
def process_folder(folder_path, excel_password):
    dataframes = {'Sheet1': [], 'Sheet2': [], 'Sheet3': []}
    files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

    for file in files:
        file_path = os.path.join(folder_path, file)
        with open(file_path, "rb") as f:
            office_file = msoffcrypto.OfficeFile(f)
            office_file.load_key(password=excel_password)
            with open("temp.xlsx", "wb") as temp_f:
                office_file.decrypt(temp_f)

        for sheet in dataframes.keys():
            df = pd.read_excel("temp.xlsx", sheet_name=sheet)
            dataframes[sheet].append(df)
    
    os.remove("temp.xlsx")
    return {sheet: pd.concat(dataframes[sheet]) for sheet in dataframes}


In [ ]:
# トップレベルのフォルダパス
top_folder_path = 'path/to/your/top/folder'
excel_password = 'your_password'

all_dataframes = {'Sheet1': [], 'Sheet2': [], 'Sheet3': []}

# トップレベルのフォルダ内の各サブフォルダに対して処理を実行
for folder in os.listdir(top_folder_path):
    folder_path = os.path.join(top_folder_path, folder)
    if os.path.isdir(folder_path):
        dfs = process_folder(folder_path, excel_password)
        for sheet in all_dataframes.keys():
            all_dataframes[sheet].append(dfs[sheet])

# 各シートのデータフレームを結合
combined_dfs = {sheet: pd.concat(all_dataframes[sheet]) for sheet in all_dataframes}

# 主キーに基づいてデータフレームを結合
final_df = combined_dfs['Sheet1']
final_df = final_df.merge(combined_dfs['Sheet2'], on=['Date', 'ISIN'], how='outer')
final_df = final_df.merge(combined_dfs['Sheet3'], on=['Date', 'ISIN'], how='outer')

print(final_df)